# add frames to training set
After training an initial network, watch videos with overlaid tracking created with `label\make_training_vid.py`. Identify frames that should be re-included in the training set (with annotations with current network's output). Use this notebook to merge these frames with an existing dataset. 

In [1]:
from deepposekit.io.utils import merge_new_images
from deepposekit.io import VideoReader
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import cv2
import os
sys.path.append(os.path.abspath(os.path.join('..')))  # add partent directory
from predict.utils import make_tracking_video
%load_ext autoreload
%autoreload 2

# settings
old_dataset = r'D:\github\fish-tracking\label\datasets\tank_dataset_5.h5'
merged_dataset = r'D:\github\fish-tracking\label\datasets\tank_dataset_6.h5'
video = r'Z:\locker\ShareData\tank_for_rick\tank_vids\20201005_Hank\concatenated.avi'

frame_nums = [21828, 24839, 30678, 32639, 63425, 63740,
              73164, 74860, 78743, 81480, 96117, 105871]
frame_nums.sort()

In [2]:
# load predictions
# (assumes video_tracking.csv already exists)
predictions = pd.read_csv(os.path.splitext(video)[0] + '_tracking.csv').to_numpy()[:,1:]
predictions = predictions.reshape((-1, int(predictions.shape[1]/3), 3))  # (frane_num X feature_num X (x,y,confidence))
keypoints = predictions[frame_nums]

I load images by scrolling through all the frames below... Directly indexing into the video results in incorrect frames being drawn from the original video. This is probably because the original (but not the tracking) video has an incorrect frame count number. I coudln't figure out how to fix this in opencv...

In [3]:
# load images
reader = VideoReader(video, batch_size=1, fast_frame_count=True)

images = []
for i in tqdm(range(frame_nums[-1]+1)):
    if i in frame_nums:
        images.append(reader.read())
    else:
        reader.read()
images = np.stack([i[...,0][...,np.newaxis] for i in images])  # replace color with singleton dimension and concatenate

# images = np.concatenate([reader[idx][...,0] for idx in frame_nums])[...,np.newaxis]  # direct indexing into the reader draws incorrect frames for mysterious reasons...

100%|█████████████████████████████████████████████████████████████████████| 105872/105872 [00:28<00:00, 3759.27it/s]


In [4]:
# make merged dataset
merge_new_images(
    datapath=old_dataset,
    merged_datapath=merged_dataset,
    images=images,
    keypoints=keypoints,
    overwrite=False
)
